### Setup

In [ ]:
# set libraries to refresh
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd
import geopandas as gpd

# import kml reading and set supported driver
import fiona
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [ ]:
from clustering.utils import pivot_by_cluster_w_stats
from utils import plot_weights_vs_radii, save_shapefiles

### Custom function for this notebook

In [ ]:
def load_and_process_barangay(selected_barangay_filepath) -> gpd.GeoDataFrame:

    cluster_gdf = gpd.read_parquet(selected_barangay_filepath)
    unique_cluster_id = (
        "B_" + cluster_gdf["PSGC"].astype(str) + "_" + cluster_gdf["cluster_id"]
    )
    cluster_gdf.insert(8, "unique_cluster_id", unique_cluster_id)

    return cluster_gdf

### Load data

In [ ]:
ROOT_DIR = Path("..")
INPUT_DATA_DIR = ROOT_DIR / "data" / "output" / "lambda_processed"
OUTPUT_DATA_DIR = ROOT_DIR / "data" / "output"
# OUTPUT_DATA_DIR = ROOT_DIR / "data_hpls" / "02_modeloutputs" / "online"

In [ ]:
# get cercle folderpaths for selected regions
filepaths = list(INPUT_DATA_DIR.glob("*"))
filepaths = filepaths[:10]

In [ ]:
# load and process all barangays
grid_gdf_list = []
for filepath in tqdm(filepaths):
    grid_gdf = load_and_process_barangay(filepath)
    grid_gdf_list.append(grid_gdf)

### Combine barangays

In [ ]:
grid_gdf = pd.concat(grid_gdf_list)
grid_gdf

In [ ]:
grid_gdf = grid_gdf.sort_values(by=["grid_id"])
grid_gdf = grid_gdf.reset_index(drop=True)
grid_gdf

### Save combined data

In [ ]:
# save grid-level dataset
save_shapefiles(grid_gdf, OUTPUT_DATA_DIR, "grids_w_clusters", formats=["parquet"])

### Pivot to cluster-level and save

In [ ]:
cluster_gdf = pivot_by_cluster_w_stats(
    gdf_w_clusters=grid_gdf,
    cluster_id_col="unique_cluster_id",
    weight_col="population",
    epsg=3121,
    cols_to_keep=[
        "PSGC",
        "urban",
        "dense_area_guess",
    ],
)

In [ ]:
import matplotlib.pyplot as plt
f, ax = plt.subplots(1, 1, figsize=(10, 10))
cluster_gdf[cluster_gdf["PSGC"]==604527012].plot(ax=ax)
grid_gdf[grid_gdf["PSGC"]==604527012].plot(markersize=1, color="black", alpha=1, ax=ax)

In [ ]:
# save cluster-level dataset
save_shapefiles(cluster_gdf, OUTPUT_DATA_DIR, "clusters", formats=["parquet"])

In [ ]:
jointplot = plot_weights_vs_radii(
    cluster_df=cluster_gdf,
    output_filepath=OUTPUT_DATA_DIR / "cluster_weights_vs_radii.png",
)